In [1]:
import requests
from bs4 import BeautifulSoup
from math import nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    return BeautifulSoup(req.text,"html.parser")

In [2]:
# url = "https://werkze.pl/jobs/48"
# page = render_html(url)

In [2]:
# gather all the data
max_item = 500
i = 16
j = 1
container = []
while len(container) < max_item:
    url = "https://werkze.pl/jobs/" + str(i)
    page = render_html(url)
    ul = page.find("ul",class_="pages pagination")
    pagination = [li.get_text() for li in ul.find_all("li")]
    container.extend(page.find_all("div",class_="item"))
    print(f"{j}:{len(container)}",end="|")
    if pagination[-1] == ">>" or pagination[-1] == ">":
        i += 16
        j += 1
    else:
        break
print("end")

1:16|2:32|3:48|4:64|5:80|6:96|7:112|8:128|9:144|10:160|11:176|12:192|13:208|14:224|15:240|16:256|17:272|18:288|19:304|20:320|21:332|end


In [3]:
jobs = []
for item in container[:500]:
    h5 = item.find("h5")
    link = h5.find("a")['href']
    title = item.find("h4").get_text().strip()
    p = item.find("p")
    wage = re.sub("\s+"," ",p.get_text())
    city = p.find_next("p").get_text().strip()
    job = {
        "Link": link,
        "Title": title,
        "Wage": wage,
        "City": city,
    }
    jobs.append(job)

In [4]:
import pandas as pd

df = pd.DataFrame(jobs)
df.to_excel("werkze.xlsx",index=False,encoding="utf-8")